In [1]:
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta


In [2]:
# create list of all yyyy-mm for data download (Note, May 2015 is the first month ND data is available in xlxs)
myList = pd.date_range('2022-04-10',datetime.today()- relativedelta(months=11), freq='MS').strftime("%Y_%m").tolist()  # with month in numeric format

# Access the ND DMR data @ https://www.dmr.nd.gov/oilgas/mprindex.asp
website = "https://www.dmr.nd.gov/oilgas/mpr/"

In [ ]:
#myList

In [41]:
NDMWD = []  #list()  # North Dakota Monthly Well Data
for month in myList:
    target_url = website + month + '.xlsx'
    print(target_url)
    r = requests.get(target_url)
    wb = pd.read_excel(r.content)
    NDMWD.append(wb)

NDMWD = pd.concat(NDMWD)

https://www.dmr.nd.gov/oilgas/mpr/2022_05.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2022_06.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2022_07.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2022_08.xlsx
https://www.dmr.nd.gov/oilgas/mpr/2022_09.xlsx


In [42]:
#Create ND monthly well data table in csv format, which can be downloaded from folder on left hand side
NDMWD.to_csv('NDMWD.csv', index=False)

In [43]:
NDMWD.tail()

,ReportDate,API_WELLNO,FileNo,Company,WellName,Quarter,Section,Township,Range,County,...,Pool,Oil,Wtr,Days,Runs,Gas,GasSold,Flared,Lat,Long
19548,2022-09-01,33089003290000,10752,"NEW MILLENNIUM RESOURCES, INC.",POLANCHEK 8-34,SENE,34,140,99,STK,...,TYLER A,528.0,3569.0,30.0,618,30.0,0,0.0,46.899267,-103.152135
19549,2022-09-01,33009005030000,2507,EMPIRE NORTH DAKOTA LLC,OBERT LINSTAD 1,SESE,22,163,79,BOT,...,SPEARFISH/MADISON,0.0,0.0,0.0,0,0.0,0,0.0,48.924446,-100.887483
19550,2022-09-01,33009012640000,6347,EMPIRE NORTH DAKOTA LLC,RICE 2,SENW,26,163,79,BOT,...,SPEARFISH/MADISON,26.0,301.0,30.0,39,0.0,0,0.0,48.917191,-100.876462
19551,2022-09-01,33009011910000,5815,EMPIRE NORTH DAKOTA LLC,RICE 23-12,NWSW,23,163,79,BOT,...,SPEARFISH/MADISON,0.0,0.0,0.0,12,0.0,0,0.0,48.928322,-100.881995
19552,2022-09-01,33009019570000,13685,EMPIRE NORTH DAKOTA LLC,RICE 3,NENW,26,163,79,BOT,...,SPEARFISH/MADISON,46.0,5979.0,28.0,73,0.0,0,0.0,48.920808,-100.877109


# **Extract first month oil and gas produciton**

In [48]:
NDMWD = pd.read_csv('NDMWD.csv')
NDMWD['ReportDate'] = pd.to_datetime(NDMWD['ReportDate'], errors = 'coerce')
NDMWD.head(2)

,ReportDate,API_WELLNO,FileNo,Company,WellName,Quarter,Section,Township,Range,County,...,Pool,Oil,Wtr,Days,Runs,Gas,GasSold,Flared,Lat,Long
0,2022-05-01,33053039010000,22023,"GRAYSON MILL OPERATING, LLC",BILL 14-23 1H,SWSW,11,151,101,MCK,...,BAKKEN,651.0,1249.0,31.0,671,4385.0,4169,180.0,47.907508,-103.580354
1,2022-05-01,33053038990000,22021,"GRAYSON MILL OPERATING, LLC",BILL 14-23 2TFH,SWSW,11,151,101,MCK,...,BAKKEN,808.0,1502.0,31.0,825,1202.0,1065,89.0,47.907450,-103.580411


In [49]:
ndWells = NDMWD[['API_WELLNO', 'Pool','ReportDate']].groupby('API_WELLNO').min().rename(columns={'ReportDate':'p0'}).reset_index()
ndWells['p1'] = ndWells.p0 + pd.DateOffset(months=1)
ndWells.drop('p0', axis=1, inplace=True )
ndWells.rename(columns={'p1':'p1Month'}, inplace=True)
ndWells.tail(3)

,API_WELLNO,Pool,p1Month
19622,33105056880000,BAKKEN,2022-10-01
19623,33105056890000,BAKKEN,2022-10-01
19624,33105057190000,CONFIDENTIAL,2022-09-01


In [50]:
ndWells = ndWells.merge(NDMWD[['API_WELLNO','Pool','ReportDate','Oil','Gas','Wtr','Days','Runs']], how='left', left_on=['API_WELLNO','Pool','p1Month'], right_on = ['API_WELLNO','Pool','ReportDate'])
ndWells.head()

,API_WELLNO,Pool,p1Month,ReportDate,Oil,Gas,Wtr,Days,Runs
0,33007000140000,HEATH,2022-06-01,2022-06-01,200.0,31.0,356.0,30.0,185.0
1,33007000260000,MADISON,2022-06-01,2022-06-01,0.0,0.0,0.0,0.0,0.0
2,33007000400000,MADISON,2022-06-01,2022-06-01,498.0,78.0,1412.0,30.0,501.0
3,33007000520000,HEATH,2022-06-01,2022-06-01,857.0,135.0,43739.0,30.0,857.0
4,33007000610000,HEATH,2022-06-01,2022-06-01,66.0,10.0,1516.0,24.0,63.0


In [51]:
ndWells.drop('ReportDate', axis=1, inplace=True )
ndWells['p1_bpd'] =  ndWells.Oil/ pd.to_datetime(ndWells['p1Month']).dt.daysinmonth
ndWells['p1_Mcfpd'] =  ndWells.Gas/ pd.to_datetime(ndWells['p1Month']).dt.daysinmonth

In [52]:
ndWells.head()

,API_WELLNO,Pool,p1Month,Oil,Gas,Wtr,Days,Runs,p1_bpd,p1_Mcfpd
0,33007000140000,HEATH,2022-06-01,200.0,31.0,356.0,30.0,185.0,6.666667,1.033333
1,33007000260000,MADISON,2022-06-01,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,33007000400000,MADISON,2022-06-01,498.0,78.0,1412.0,30.0,501.0,16.600000,2.600000
3,33007000520000,HEATH,2022-06-01,857.0,135.0,43739.0,30.0,857.0,28.566667,4.500000
4,33007000610000,HEATH,2022-06-01,66.0,10.0,1516.0,24.0,63.0,2.200000,0.333333


In [53]:
ndWells.to_csv('ndWells.csv', index=False)